## Evaluating classification techniques for speaker characterization
### Laura Fernández Gallardo

After evluating the binary classification of speakers' warmth-attractiveness (WAAT), I examine in this notebook multilabel classification, that is, predicting several traits attributed to speakers, which are not mutually exclusive.   

* For each perceptive speaker interpersonal dimension generated from [factor analysis](https://github.com/laufergall/Subjective_Speaker_Characteristics/tree/master/speaker_characteristics/factor_analysis) thrsholding scores based on percentiles to define 3 classes ("high", "mid", and "low") with approximately the same number of samples. These dimensions are: *warmth*, *attractiveness*, *confidence*, *compliance*, and *maturity*.
* Only the "high" and "low" classes are of interest -> I address **multilabel binary classification**.
* As evaluation metric, I will consider the average per-class accuracy (average of sensitivity and specificity)